# 데이터시각화특강 기말평가 답안 (문제1)
## 202055364 황성윤

## 1. COVID19 시도별 접종률 시각화 (30점)
아래의 그림은 COVID19 예방접종의 시도별 현황을 캡쳐한 것이다.

이 정보는 특정 주기로 업데이트 되며 아래의 웹페이지 2번째 테이블에서 확인할 수 있다.

https://ncv.kdca.go.kr/mainStatus.es?mid=a11702000000

판다스의 `pd.read_html()` 함수를 이용해 위의 페이지에서 그림1의 테이블을 읽어오라. 그리고 folium의 choroplethmap을 활용하여 시도별 2차접종의 접종률을 시각화 하라. 시각화 예시는 그림2와 같다.

In [1]:
import pandas as pd
import numpy as np
import folium
import json
import requests

In [2]:
covid_data=pd.read_html('https://ncv.kdca.go.kr/mainStatus.es?mid=a11702000000',encoding='utf-8')
covid19=covid_data[1]
covid19

구분  1차접종            2차접종            3차접종         
    구분 당일 실적     당일 누계 당일 실적     당일 누계 당일 실적    당일 누계
0   합계  1784  42958480  1308  41687101  7772  6355419
1   서울   304   7994449   284   7782371  1826  1199359
2   부산    81   2746932    74   2661984   395   393801
3   대구    37   1911532    25   1850943   152   237143
4   인천   138   2451153    89   2377704   525   343380
5   광주    20   1199833    14   1160092   102   172612
6   대전    45   1183193    23   1145554   190   163116
7   울산    22    916099    14    888799    96   103671
8   세종    24    275242     5    265119    47    36185
9   경기   765  11249307   489  10919282  3096  1566530
10  강원    32   1288650    22   1252101   123   215955
11  충북    46   1361335    29   1322178   277   214037
12  충남    70   1803008    85   1746430   295   294267
13  전북    36   1525810    23   1483651   139   280951
14  전남    25   1585352    42   1539412   102   317220
15  경북    44   2175725    30   2107914   116   337140
16  경남    56   2735495    44   2647507   188   406192
17  제주    39    555365    16    536060   103    73860

In [3]:
covid19_1=covid19.iloc[1:,:].reset_index(drop=True)
covid19_1

구분  1차접종            2차접종            3차접종         
    구분 당일 실적     당일 누계 당일 실적     당일 누계 당일 실적    당일 누계
0   서울   304   7994449   284   7782371  1826  1199359
1   부산    81   2746932    74   2661984   395   393801
2   대구    37   1911532    25   1850943   152   237143
3   인천   138   2451153    89   2377704   525   343380
4   광주    20   1199833    14   1160092   102   172612
5   대전    45   1183193    23   1145554   190   163116
6   울산    22    916099    14    888799    96   103671
7   세종    24    275242     5    265119    47    36185
8   경기   765  11249307   489  10919282  3096  1566530
9   강원    32   1288650    22   1252101   123   215955
10  충북    46   1361335    29   1322178   277   214037
11  충남    70   1803008    85   1746430   295   294267
12  전북    36   1525810    23   1483651   139   280951
13  전남    25   1585352    42   1539412   102   317220
14  경북    44   2175725    30   2107914   116   337140
15  경남    56   2735495    44   2647507   188   406192
16  제주    39    555365    16    536060   103    73860

In [4]:
df=pd.read_csv('https://raw.githubusercontent.com/guebin/2021DV/master/_notebooks/2021-11-22-prov.csv')
df

,행정구역(시군구)별,총인구수 (명)
0,서울특별시,9532428
1,부산광역시,3356311
2,대구광역시,2390721
3,인천광역시,2945009
4,광주광역시,1442454
5,대전광역시,1454228
6,울산광역시,1122566
7,세종특별자치시,368276
8,경기도,13549577
9,강원도,1537717


In [5]:
df2=pd.DataFrame({'province':df.iloc[:,0],'total':df.iloc[:,1],'vaccine':covid19_1.iloc[:,4]})
df2['proportion']=df2['vaccine']/df2['total']
df2

,province,total,vaccine,proportion
0,서울특별시,9532428,7782371,0.816410
1,부산광역시,3356311,2661984,0.793128
2,대구광역시,2390721,1850943,0.774220
3,인천광역시,2945009,2377704,0.807367
4,광주광역시,1442454,1160092,0.804249
5,대전광역시,1454228,1145554,0.787740
6,울산광역시,1122566,888799,0.791757
7,세종특별자치시,368276,265119,0.719892
8,경기도,13549577,10919282,0.805876
9,강원도,1537717,1252101,0.814260


In [6]:
global_distriction_jsonurl='https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-provinces-2018-geo.json'
global_dict = json.loads(requests.get(global_distriction_jsonurl).text)
[global_dict['features'][i]['properties']['name'] for i in range(17)]

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [7]:
import plotly.express as px
from IPython.display import HTML

In [8]:
m = folium.Map([36,128],zoom_start=7,scrollWheelZoom=False)
choro = folium.Choropleth(
    data=df2, 
    geo_data=global_dict,
    columns=['province','proportion'],
    key_on = 'feature.properties.name'
)
choro.add_to(m) 
#m